# Explore the data

We will lated come back with the test data. So just work on df_ed and df_hh
GOAL: get a good prediction as quickly as possible
Don'e do the "Hh  poverty score continuation' for now

In [5]:
import pandas as pd
import os, sys
import matplotlib.pyplot as plt


In [6]:
# Additional covariate called family size. to USE more data. 
# How to use more data by groupping poverty_scores on household?
# within same household
# add a psu_hh uui as a VARIABLE to tell the model "predict something similar for members of the same family". It would be difficult, there is a lot.
# stacking? what is stacking?

# Root preprocessing:

In [8]:
ed = pd.read_csv('ed_train_raw.csv') # education DataFrame
hh = pd.read_csv('hh_train_raw.csv') # household DataFrame



In [9]:
### Prepare Ed data
def prepare_df(df, suffix:str):
    uids = df['psu'].astype(str) + "_"  + df['hh'].astype(str) + "_" + df['idcode'].astype(str) 
    df = df.drop(columns=['psu', 'hh', 'idcode'])
    df.columns = [suffix + "_" + col.capitalize() for col in df.columns]
    df.insert(0, 'uid', uids)
    return df

ed = prepare_df(ed, 'ED')
hh = prepare_df(hh, 'HH')

# Remove absoluteley useless variates

In [11]:
hh = hh.drop(columns=['HH_Hhid', 'HH_Q04', 'HH_Q08', 'HH_Q12', 'HH_Q18'])
columns_to_keep = [
    'uid',
    'ED_Q01', 'ED_Q02', 'ED_Q03', 'ED_Q04', 'ED_Q05', 'ED_Q06', 'ED_Q07', 'ED_Q08', 'ED_Q09', 'ED_Q10', 'ED_Q11', 
    'ED_Q14', 'ED_Q15', 'ED_Q16', 'ED_Q17', 'ED_Q18',
    'ED_Q19',
    'ED_Q23',
    'ED_Q26', 'ED_Q27', 'ED_Q28', 'ED_Q29',
    'ED_Q41', 
]

ed = ed[columns_to_keep]


# Filter data based on uids that have a poverty score assigned to them.

### This step is ONLY for TRAIN data. 

In [13]:
poverty = pd.read_csv('poverty_train_raw.csv') # poverty/answers

### compute poverty scores
for i in range(1,11):
    col = 'subjective_poverty_'+ str(i)
    poverty.loc[poverty[col]==1, 'poverty_score'] = i

poverty['uid'] = poverty['psu_hh_idcode']
poverty = poverty[['uid', 'poverty_score']]


ed = ed[ed['uid'].isin(poverty['uid'])]
ed = ed.merge(poverty[['uid', 'poverty_score']], on='uid', how='left')

hh = hh[hh['uid'].isin(poverty['uid'])]
hh = hh.merge(poverty[['uid', 'poverty_score']], on='uid', how='left')




In [14]:
# # quite a lot of data wasted... nothing we can do.... right?
# print(train_ed.shape)
# print(train_hh.shape)
# print(poverty.shape)

# # Explore test data:
# print(test_ed.shape)
# print(test_hh.shape)
# assert(test_hh[test_hh['uid'].isin(test_ed['uid'])].shape[0] == 1334)
# ## No issues here. just use them as they are

In [15]:
# df_hh.to_csv('hh_train_transform.csv', index=False)
# df_ed.to_csv('ed_train_transform.csv', index=False)
# df_pov_scores.to_csv('scores_train_transform.csv', index=False)


# Rename variates for interpretability

In [17]:
# ED variates
ed['literate'] = ed['ED_Q01'].eq(1) & ed['ED_Q02'].eq(1) 

ed['attended_school'] = ed['ED_Q03'] 

ed['highest_school_lvl'] = ed['ED_Q04'] 

ed['preschool'] = ed['ED_Q07']

# Merge
ed['now_enrolled'] = ed['ED_Q08']
ed['now_attending'] = ed['ED_Q09'] # 1 yes, 2 No
ed['past_enrolled'] = ed['ED_Q14'] # 1 yes, 2 No
ed['past_attending'] = ed['ED_Q15'] # 1 yes, 2 No

# Merge
ed['now_not_attend_reason'] = ed['ED_Q10']
ed['now_not_enroll_reason'] = ed['ED_Q11']
ed['past_not_attend_reason'] = ed['ED_Q16'] 
ed['past_not_enroll_reason'] = ed['ED_Q17']

ed['finish_school_age'] = ed['ED_Q18'] 

ed['younger_19'] = ed['ED_Q19']

ed['public_private_school'] = ed['ED_Q23'] 

ed['school_transportation_time'] = ed['ED_Q26'].apply(lambda hrs : hrs * 60) +  ed['ED_Q27'] # it works
ed['school_transportation_vehicle'] = ed['ED_Q28'] 
ed['school_transportation_cost'] = ed['ED_Q29'] 

ed['school_expenses'] = ed['ED_Q41'] 


In [18]:
new_cols = [
    'uid',
    'literate','attended_school','highest_school_lvl',
    'preschool','now_enrolled','now_attending','past_enrolled', 'past_attending','now_not_attend_reason',
    'now_not_enroll_reason','past_not_attend_reason','past_not_enroll_reason','finish_school_age',
    'younger_19','public_private_school','school_transportation_time','school_transportation_vehicle',
    'school_transportation_cost','school_expenses',
    'poverty_score'
]


ed = ed[new_cols]


In [19]:

# HH Variates. A random forest might atually be good
hh['sex'] = hh['HH_Q02'] 
hh['family_role'] = hh['HH_Q03'] 
hh['age'] = hh['HH_Q05y'] # don't need month decimal
hh['marital_status'] = hh['HH_Q06'] 
hh['spouse_lives'] = hh['HH_Q07']
hh['time_away'] = hh['HH_Q09'] 
# hh[''] = hh['HH_Q10'] 

# leave the sorting to the tree. that's what ti does better than anybody!!!! 
hh['lives_with_mom'] = hh['HH_Q11'] 
hh['moms_education'] = hh['HH_Q13'] 
hh['mom_alive'] = hh['HH_Q14'] 

hh['lives_with_dad'] = hh['HH_Q17'] 
hh['dad_education'] = hh['HH_Q19'] 
hh['dad_alive'] = hh['HH_Q20']

hh = hh.drop(columns=['HH_Q02', 'HH_Q03', 'HH_Q05y', 'HH_Q05m', 'HH_Q06', 'HH_Q07',
       'HH_Q09', 'HH_Q10', 'HH_Q11', 'HH_Q13', 'HH_Q14', 'HH_Q15', 'HH_Q16',
       'HH_Q17', 'HH_Q19', 'HH_Q20', 'HH_Q21', 'HH_Q22'])


# Group #1 

Claim: Those who answer No = 2 for attended_school are done with the ED survey. 


#### Group 1 only uses ED 1,2




In [30]:
# get group_1 train sets
g1condition = ed['attended_school'].eq(2)
g1ed = ed[condition]

g1ed = g1ed[['uid', 'literate']]
g1hh = hh[hh['uid'].isin(g1ed['uid'])]

In [23]:
# # Save them to /group1
# g1ed.to_csv('./group1/ed_train_g1.csv', index=False)
# g1hh.to_csv('./group1/hh_train_g1.csv', index=False)


# Group 2

The ones done with studies

In [57]:
split = ed.loc[~g1condition]
# Try to literally look for those who have completed school (Option 13 in either 11 or 17)
g2condition = split['now_not_enroll_reason'].eq(13) | split['past_not_enroll_reason'].eq(13)

g2ed = split[g2condition]
g2hh = hh[hh['uid'].isin(g2ed['uid'])]

print(g2ed.shape)
print(g2hh.shape)

(3594, 21)
(3594, 14)


In [63]:
g2ed.to_csv('./group2/ed_train_g2.csv', index=False)
g2hh.to_csv('./group2/hh_train_g2.csv', index=False)


# G3

#### The students

Should not use certain ed questions in their prediction... the ones meant for students doi! determine which ones these are

In [69]:
g3ed = ed.loc[~g1condition & ~g2condition]
g3hh = hh[hh['uid'].isin(g3ed['uid'])]
print(g3df.shape)
print(g3hh.shape)

(1601, 21)
(1601, 14)


In [71]:
g3ed.to_csv('./group3/ed_train_g3.csv', index=False)
g3hh.to_csv('./group3/hh_train_g3.csv', index=False)
